<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Marc Spiegelman, Template from Kyle Mandli</td>
</table>


# Fo Stixrude behavior

This notebook will use functions from the ThermoCodegen generated Fo-stixrude  and compare output to objective C code in thermoengine.  There seems to be a something in Stixrude (but not berman) end-members which is causing endmembers to report different values after different initialization.  Also going to check phases

Suspicion is that Volume is a hidden variable that is not updating properly

In [1]:
# load the standard goodies
import numpy as np
from scipy.optimize import fsolve, brentq
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline
plt.rcParams["figure.figsize"] = [8,6]
plt.rcParams['font.size'] = 16

### Load the ThermoCodegen object for the Stixrude Mg2SiO4 system

This will require that the python bindings for the py_fo_fa_binary module are in the python path

if you are using modules,  this can be accomplished by running
|
```bash
cd ../../reactions/fo_fa_binary
module load ./fo_fa_binary.module


In [2]:
import py_Mg2SiO4_stixrude as pms
pms.phase_info()

{'Abbrev': ['Fa', 'Ol', 'Wa', 'FeWa', 'FeRi', 'Ri', 'MgRi', 'MgWa', 'Fo'],
 'ClassName': ['Fayalite',
  'Olivine',
  'Wadsleyite',
  'FeWadsleyite',
  'FeRingwoodite',
  'Ringwoodite',
  'MgRingwoodite',
  'MgWadsleyite',
  'Forsterite'],
 'PhaseType': ['pure',
  'solution',
  'solution',
  'pure',
  'pure',
  'solution',
  'pure',
  'pure',
  'pure']}

In [3]:
rxn = pms.Mg2SiO4_stixrude()
rxn.report()

Phase 0 Forsterite
     Endmember 0 Forsterite_stixrude
Phase 1 MgWadsleyite
     Endmember 0 MgWadsleyite_stixrude
Phase 2 MgRingwoodite
     Endmember 0 MgRingwoodite_stixrude


In [4]:
#from thermoengine import model
#te = model.Database(database='Stixrude')
#te.phase_info


utility functions 


In [5]:
toCelsius = lambda T: T - 273.15
toKelvin = lambda T: T + 273.15
toBar = lambda P: P*10000.

### Really need to sort out the State issue

It seems like the values returned on at least tcg stixrude models maintain some state and give different answers depending on how the module was first called.

First I want to isolate this problem,  then check to see if the Berman models have the same issues


In [6]:
P=toBar(10.)
#Ta = np.array([ 1700., 2200., 2500.])
Ta = np.array([ 2200., 2500., 1700])
#Ta = np.array([2500., 1700., 2200.])
c = [ 1. ]

In [7]:
Fo = pms.Forsterite_stixrude()
Fo_phase = pms.Forsterite()

print('Ta = {}'.format(Ta))
print('T,\t Volume,\tendmember,\t phase')
for T in Ta:
    print(T, Fo.v(T,P),Fo.g(T, P), Fo_phase.g(T,P,c),)

Ta = [2200. 2500. 1700.]
T,	 Volume,	endmember,	 phase
2200.0 4.264748128775319 -2175345.725191568 -2175345.725191568
2500.0 4.303347056870189 -2300256.196764278 -2300256.196764278
1700.0 4.206004728741926 -1984446.1221989086 -1984446.1221989086


#### 1700 first with clean kernel restart

Old version with overly caching solve_V routine
```
Ta = [1700. 2200. 2500.]
T,	 Volume,	endmember,	 phase
1700.0 4.206004728741926 -1984446.1221989086 -1984446.1221989086
2200.0 4.206004728741926 -2174796.0112393694 -2174796.0112393694
2500.0 4.206004728741926 -2298809.9893216193 -2298809.9893216193
```

New version with changing Volumes
```
Ta = [1700. 2200. 2500.]
T,	 Volume,	endmember,	 phase
1700.0 4.206004728741926 -1984446.1221989086 -1984446.1221989086
2200.0 4.26474812877532 -2175345.725191568 -2175345.725191568
2500.0 4.303347056870189 -2300256.196764278 -2300256.196764278
```

#### 2200 first with clean kernel restart

Old version with overly caching solve_V routine
```
Ta = [2200. 2500. 1700.]
T,	 Volume,	endmember,	 phase
2200.0 4.264748128775319 -2175345.725191568 -2175345.725191568
2500.0 4.264748128775319 -2300034.733225503 -2300034.733225503
1700.0 4.264748128775319 -1983874.101775966 -1983874.101775966
```

New version with changing Volumes
```
Ta = [2200. 2500. 1700.]
T,	 Volume,	endmember,	 phase
1700.0 4.206004728741926 -1984446.1221989086 -1984446.1221989086
2200.0 4.264748128775319 -2175345.725191568 -2175345.725191568
2500.0 4.303347056870189 -2300256.196764278 -2300256.196764278
```

#### 2500 first with clean kernel restart
```
Ta = [2500. 1700. 2200.]
T,	 endmember,	 phase
1700.0 -1982900.6402404727 -1982900.6402404727
2200.0 -2175118.282576279 -2175118.282576279
2500.0 -2300256.196764278 -2300256.196764278
```

the back to original order (without kernel restart)
```
Ta = [1700. 2200. 2500.]
T,	 endmember,	 phase
1700.0 -1982900.6402404727 -1982900.6402404727
2200.0 -2175118.282576279 -2175118.282576279
2500.0 -2300256.196764278 -2300256.196764278
```

### Repeat with Berman Forsterite

Which doesn't show this state affect

In [8]:
import py_fo_fa_binary as pb
pb.phase_info()

ModuleNotFoundError: No module named 'py_fo_fa_binary'

In [9]:
rxn_fofa = pb.fo_fa_binary()
rxn_fofa.report()

NameError: name 'pb' is not defined

In [10]:
# repeat for berman

Fo_b = pb.Forsterite_berman()
print('Ta = {}'.format(Ta))
for T in Ta:
    print(T, Fo_b.g(T, P))

NameError: name 'pb' is not defined

#### 1700 first with clean kernel restart
```
Ta = [1700. 2200.]
1700.0 -2125970.794519817
2200.0 -2312664.014428501
```

#### 2200 first with clean kernel restart
```
Ta = [2200. 1700.]
1700.0 -2125970.794519817
2200.0 -2312664.014428501
```

In [ ]:
'''
Fo_te = te.get_phase('Fo')
Fa_te = te.get_phase('Fa')
Wa_te = te.get_phase('MgWds')

print(Fo_te.gibbs_energy(T,P))
print(Fa_te.gibbs_energy(T,P))
print(Wa_te.gibbs_energy(T,P))
'''

In [ ]:
Fo = pms.Forsterite_stixrude()
Fa = pms.Fayalite_stixrude()
Wa = pms.MgWadsleyite_stixrude()

print(T,P)
print(Fo.g(T,P))
#print(Fa.g(T,P))
#print(Wa.g(T,P))
print(Fo.g(2000.,P), Fo.g(2500.,P))

* T,P = 2500.0 100000.0
* -2300256.196764278
* -2095583.760392555 -2300256.196764278

* T,P = 2000.0 100000.0
* -2096204.0149583982
* -2096204.0149583982 -2299662.150856483

In [ ]:
err_Fo = lambda T,P: (Fo_te.gibbs_energy(T,P) - Fo.g(T,P))/Fo_te.gibbs_energy(T,P)
err_Fa = lambda T,P: (Fa_te.gibbs_energy(T,P) - Fa.g(T,P))/Fa_te.gibbs_energy(T,P)
err_Wa = lambda T,P: (Wa_te.gibbs_energy(T,P) - Wa.g(T,P))/Wa_te.gibbs_energy(T,P)
print(err_Fo(T,P), err_Fa(T,P), err_Wa(T,P))

compare affinities along a T,P trajectory first


In [ ]:
T=2000.
P = np.linspace(.1,2)
Pbar = toBar(P)
G_Fo_te = np.array([ Fo_te.gibbs_energy(T,p) for p in Pbar])
G_Fo =  np.array([ Fo.g(T,p) for p in Pbar])
G_Fa_te = np.array([ Fa_te.gibbs_energy(T,p) for p in Pbar])
G_Fa =  np.array([ Fa.g(T,p) for p in Pbar])



In [ ]:
fig = plt.figure(figsize=(8,6))
axes = fig.add_subplot(1,1,1)
axes.plot(P, (G_Fo_te - G_Fo)/(G_Fo_te) ,'r',label='Fo')
axes.plot(P, (G_Fa_te - G_Fa)/(G_Fa_te) ,'g',label='Fa')
axes.legend(loc='best')
axes.set_xlabel('P (Gpa)')
axes.set_ylabel('rel_error TE-TCG')
axes.set_title('T = {}'.format(T))
axes.grid()
plt.show()

### Plot the Error Surfaces Fo and Fa

In [ ]:
ta = np.linspace(1000.,2500.,100)
pa = np.linspace(.1,2.5,100)
Tm,Pm = np.meshgrid(ta,pa)
rerr_Fo = np.zeros(Tm.shape)
rerr_Fa = np.zeros(Tm.shape)

for i,T in enumerate(ta):
    for j,P in enumerate(toBar(pa)):
        rerr_Fo[j,i] = err_Fo(T,P)
        rerr_Fa[j,i] = err_Fa(T,P)

## Error plot Diagram

Contour 0 affinity for each reaction

In [ ]:
fig = plt.figure(figsize=(20, 6))
axes = fig.add_subplot(1,2,1)
cplot_fo = axes.contourf(Pm,Tm,rerr_Fo)
plt.colorbar(cplot_fo)

axes.set_title('Fo')

axes = fig.add_subplot(1,2,2)
cplot_fa = axes.contourf(Pm,Tm,rerr_Fa)
plt.colorbar(cplot_fa)

axes.set_title('Fa')

axes.grid()
axes.set_xlabel('Pressure (GPa)')
axes.set_ylabel('Temperature (K)')
plt.show()